# ISIC Archive

In [1]:
import girder_client
import pandas as pd
import numpy as np
import json
import requests
import urllib.request
import progressbar

In [2]:
gc = girder_client.GirderClient(apiUrl='https://isic-archive.com/api/v1')
dataset_df = pd.DataFrame(gc.get("/dataset"))
dataset_names = list(dataset_df["name"])
dataset_names

['ISIC_MSK-1_1',
 'ISIC_MSK-1_2',
 'ISIC_MSK-2_1',
 'ISIC_MSK-3_1',
 'ISIC_MSK-4_1',
 'ISIC_MSK-5_1',
 'ISIC_SONIC_1',
 'ISIC_UDA-1_1',
 'ISIC_UDA-2_1']

## Get Image Metadata

Choosing the database for which the file metadata should be saved.

It seems like we could use the following:
- MSK: 11, 12, 21, 41
- uda: 11 and 21

In [8]:
path_image = "/image/"
path_dest = "/home/dwx/tmp/"
limit = 2 # 0 no limit, x>0 limit x

db_selection = [0,1,2,4,7,8]
db_selection = range(len(dataset_names))
sel_dataset_names = [dataset_names[a] for a in db_selection]
sel_dataset_names

['ISIC_MSK-1_1',
 'ISIC_MSK-1_2',
 'ISIC_MSK-2_1',
 'ISIC_MSK-3_1',
 'ISIC_MSK-4_1',
 'ISIC_MSK-5_1',
 'ISIC_SONIC_1',
 'ISIC_UDA-1_1',
 'ISIC_UDA-2_1']

In [9]:
def getSpecificMetadata(curId, jsonData):
    #result += [jsonData["meta"]["clinical"]["benign_malignant"]]
    result = {}
    result = jsonData["meta"]["clinical"]
    result["name"] = jsonData["name"]
    result["id"] = curId
    return result

for a in progressbar.log_progress(sel_dataset_names[:]):
    cur_dataset_id = dataset_df[dataset_df["name"] == a]._id
    cur_dataset_id = cur_dataset_id.tolist()[0]
    
    cur_selection = gc.get(path_image,parameters={"datasetId":cur_dataset_id, "limit":0})[:limit]
    result = []
    for b in progressbar.log_progress(cur_selection):
        cur_id = b["_id"]
        curMetadata = gc.get(path_image+str(cur_id),parameters={})
        specificMetadata = getSpecificMetadata(cur_id, curMetadata)
        result += [specificMetadata]
    result_df = pd.DataFrame(result)
    result_df.to_csv(path_dest+a+".csv")